In [1]:
!pip install transformers
!pip install peft
!pip install torch
!pip install datasets


Import the necessary libraries

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset, DatasetDict

import os
os.environ["WANDB_DISABLED"] = "true"

Log into Hugging Face with a secure token

In [3]:
from google.colab import userdata
from huggingface_hub import login

# Retrieve the Hugging Face token securely
hf_token = userdata.get("HF_TOKEN")  # Replace 'HF_TOKEN' with your secret name

if not hf_token:
    raise ValueError("Hugging Face token not found in Colab secrets.")

# Log in using the token
login(token=hf_token)
print("Successfully logged into Hugging Face!")

Successfully logged into Hugging Face!


Create Some Instruction Training Examples to Fine-Tune the Model

In [5]:
examples = []

In [45]:
examples = [
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I haven't received my order #12345 yet. It's been a week.",
        "output": """Thank you for contacting TechCorp Support.

Order Status: Delayed
Reference: #12345
Action Taken: Order status check initiated

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I received the wrong item in my order #67890.",
        "output": """Thank you for contacting TechCorp Support.

Order Status: Wrong Item Delivered
Reference: #67890
Action Taken: Replacement item initiated

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "My order #98765 is damaged. What should I do?",
        "output": """Thank you for contacting TechCorp Support.

Order Status: Damaged Item
Reference: #98765
Action Taken: Return and replacement process initiated

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I want to cancel my order #54321.",
        "output": """Thank you for contacting TechCorp Support.

Order Status: Cancellation Request
Reference: #54321
Action Taken: Cancellation process initiated

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I was charged twice for my order #11223. Please help.",
        "output": """Thank you for contacting TechCorp Support.

Order Status: Billing Issue
Reference: #11223
Action Taken: Refund request initiated

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "When will my order #44556 arrive?",
        "output": """Thank you for contacting TechCorp Support.

Order Status: In Transit
Reference: #44556
Action Taken: Estimated delivery date provided

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "My order #99988 was returned. When will I get my refund?",
        "output": """Thank you for contacting TechCorp Support.

Order Status: Returned
Reference: #99988
Action Taken: Refund process initiated

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I can't track my order #66777. Can you help?",
        "output": """Thank you for contacting TechCorp Support.

Order Status: Tracking Issue
Reference: #66777
Action Taken: Tracking link resent

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support"""
    },
        {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "My order #56789 was supposed to arrive yesterday, but I haven't received it.",
        "output": """Thank you for contacting Tech Support.

Order Status: Delayed
Reference: #56789
Action Taken: Initiated a follow-up with the shipping carrier

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
Tech Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I received a damaged item in my order #98765. What should I do?",
        "output": """Thank you for contacting Tech Support.

Order Status: Damaged Item
Reference: #98765
Action Taken: Replacement process initiated. Return instructions sent.

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
Tech Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I was charged twice for my order #11223. Please help.",
        "output": """Thank you for contacting Tech Support.

Order Status: Billing Issue
Reference: #11223
Action Taken: Refund process initiated for duplicate charge

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
Tech Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "When will my order #33445 arrive?",
        "output": """Thank you for contacting Tech Support.

Order Status: In Transit
Reference: #33445
Action Taken: Estimated delivery date provided via email

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
Tech Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I need to cancel my order #44556 immediately.",
        "output": """Thank you for contacting Tech Support.

Order Status: Cancellation Requested
Reference: #44556
Action Taken: Cancellation process initiated. Confirmation email sent.

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
Tech Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I can't track my order #77889. The tracking link doesn't work.",
        "output": """Thank you for contacting Tech Support.

Order Status: Tracking Issue
Reference: #77889
Action Taken: Sent updated tracking link. Contacted carrier for more information.

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
Tech Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I returned my order #99000 last week. When will I get my refund?",
        "output": """Thank you for contacting Tech Support.

Order Status: Returned
Reference: #99000
Action Taken: Refund process initiated. Refund expected within 5 business days.

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
Tech Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "My order #11122 arrived, but it's missing some items.",
        "output": """Thank you for contacting Tech Support.

Order Status: Incomplete Order
Reference: #11122
Action Taken: Missing items identified and replacement initiated

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
Tech Support"""
    }
]


Format our training examples the way we want (Instruction, Input, Response)

In [46]:
dataset = Dataset.from_list(examples)

In [47]:
def format_example(example):
    instruction = example["instruction"]
    input_text = example["input"]
    output = example["output"]
    return {
        "text": f"### Instruction: {instruction}\n### Input: {input_text}\n### Response: {output}"
    }

formatted_examples = [format_example(ex) for ex in examples]
dataset = Dataset.from_list(formatted_examples)

Load the tokenizer

In [48]:
from transformers import AutoTokenizer

model_name = "meta-llama/Llama-3.2-1B-Instruct"  # Adjust to an available model
model = AutoModelForCausalLM.from_pretrained(model_name, token=os.getenv("HF_TOKEN"))

tokenizer = AutoTokenizer.from_pretrained(model_name, token=os.getenv("HF_TOKEN"))

print("Tokenizer loaded successfully!")

Tokenizer loaded successfully!


In [49]:
# Ensure tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use eos_token if available
    # Or add a new pad token:
    # tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [50]:
def tokenize_function(examples):
    # The dataset now only contains the 'text' column
    tokenized = tokenizer(
        examples['text'], # Access the 'text' column directly
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )
    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)


# Inspect the tokenized dataset
print(tokenized_dataset.column_names)  # Should include 'input_ids' and 'attention_mask'
print(tokenized_dataset)

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

['text', 'input_ids', 'attention_mask', 'labels']
Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 16
})


In [51]:
# LoRA configuration
lora_config = LoraConfig(
    r=16,  # Rank of the low-rank adaptation matrix. Lower values reduce memory usage.
    lora_alpha=32,  # Scaling factor for LoRA updates. Higher values increase capacity.
    target_modules=["q_proj", "v_proj"],  # Ensure these modules exist in your model.
    lora_dropout=0.1,  # Dropout rate to prevent overfitting.
    bias="none",  # Do not adapt biases in the model.
    task_type="CAUSAL_LM"  # Task type: causal language modeling.
)

model = get_peft_model(model, lora_config)

In [52]:
training_args = TrainingArguments(
    output_dir="./results",  # Directory where model checkpoints and logs will be saved
    num_train_epochs=30,  # Number of complete passes through the training dataset
    learning_rate=1e-4,  # Initial learning rate for the optimizer
    per_device_train_batch_size=4,  # Batch size per GPU/CPU during training
    per_device_eval_batch_size=4,  # Batch size per GPU/CPU during evaluation
    warmup_steps=50,  # Number of steps for the learning rate warmup phase
    weight_decay=0.01,  # Strength of weight decay for regularization
    logging_steps=30,  # Log training metrics every 10 steps
    eval_strategy="steps",  # Evaluation strategy: perform evaluation every `eval_steps`
    eval_steps=20,  # Perform evaluation every 20 steps
    save_strategy="steps",  # Save model checkpoints every `save_steps`
    save_steps=20,  # Save model checkpoint every 20 steps
    load_best_model_at_end=True,  # Automatically load the best model at the end of training
    metric_for_best_model="loss",  # Metric to determine the best model (in this case, lowest loss)
    report_to="none"  # Disable integration with external logging tools like W&B
)

In [53]:
# Split the dataset into train (80%) and validation (20%)
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)

# Convert to DatasetDict format
tokenized_dataset = DatasetDict({
    "train": split_dataset["train"],
    "validation": split_dataset["test"],
})

# Verify the structure
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 12
    })
    validation: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4
    })
})


In [54]:
model.print_trainable_parameters()

trainable params: 1,703,936 || all params: 1,237,518,336 || trainable%: 0.1377


In [55]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],      # Training dataset
    eval_dataset=tokenized_dataset["validation"], # Validation dataset
    processing_class=tokenizer,                           # Tokenizer
)

In [56]:
# Step 3: Train the model
trainer.train()

# Save the final model
trainer.save_model("./final_model")
print("Training complete and model saved!")

Step,Training Loss,Validation Loss
20,No log,3.390074
40,4.433400,0.468085
60,0.392600,0.187010
80,0.392600,0.124155


Training complete and model saved!


Inference function

In [57]:
def generate_response(model, tokenizer, instruction, input_text=""):
    """
    Generate a response using the fine-tuned model.

    Args:
        model: The fine-tuned model
        tokenizer: The tokenizer
        instruction: The instruction for the model
        input_text: Optional input text

    Returns:
        str: The generated response
    """
    # Format input to match training format
    prompt = f"### Instruction: {instruction}\n### Input: {input_text}\n### Response:"

    # Tokenize the prompt
    inputs = tokenizer(prompt,
                      return_tensors="pt",
                      truncation=True,
                      max_length=512,
                      add_special_tokens=True)

    # Move inputs to the same device as model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Generate response
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,  # Adjust based on desired response length
        num_return_sequences=1,
        temperature=0.1,     # Adjust for response creativity (0.0-1.0)
        do_sample=True,
        top_p=0.95,         # Nucleus sampling
        top_k=50,           # Top-k sampling
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the response part (after "### Response:")
    response_parts = response.split("### Response:")
    if len(response_parts) > 1:
        response = response_parts[1].strip()

    return response


In [58]:
def post_process_response(response):
    """
    Post-process the model's response to ensure it adheres to the desired format.

    Args:
        response (str): The raw response from the model.

    Returns:
        str: The post-processed response with required sections.
    """
    # Define required sections
    sections = ["Order Status", "Reference", "Action Taken", "Need immediate assistance?"]
    processed_response = response.strip()

    # Ensure all required sections exist
    for section in sections:
        if section not in processed_response:
            processed_response += f"\n{section}: [Details Missing]"

    return processed_response



In [59]:
# Example input
instruction = "Respond to this customer inquiry following our format"
input_text = "I haven't received my order #12345 yet. It's been a week."

# Generate response
raw_response = generate_response(model, tokenizer, instruction, input_text)

# Post-process the response
final_response = post_process_response(raw_response)

# Print the results
print("\nINSTRUCTION:")
print(instruction)
print("\nINPUT:")
print(input_text)
print("\nFINAL RESPONSE:")
print(final_response)


INSTRUCTION:
Respond to this customer inquiry following our format

INPUT:
I haven't received my order #12345 yet. It's been a week.

FINAL RESPONSE:
Thank you for contacting TechCorp Support.

Order Status: Delayed
Reference: #12345
Action Taken: Tracking update sent
Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support


In [ ]:
# Check training logs/loss
print("Training logs from the last few steps:")
print(trainer.state.log_history[-5:])  # Shows the last 5 training logs

Training logs from the last few steps:
[{'train_runtime': 9.3961, 'train_samples_per_second': 4.257, 'train_steps_per_second': 1.064, 'total_flos': 119789573898240.0, 'train_loss': 2.6833833694458007, 'epoch': 10.0, 'step': 10}]
